In [8]:
import gym
import random
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from keras import backend as K
import numpy as np

In [9]:

def initiate_NN(learning_rate=1e-3):
    model = Sequential()
    model.add(Dense(64, input_dim=8, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(4, activation='linear'))
    model.compile(loss='mse', optimizer=Adam(lr=learning_rate))
    return model

def epsilon_action(epsilon):
    random_value = np.random.random()
    if random_value<=epsilon:
        action = env.action_space.sample()
    else:
        Y=online_model.predict(state.reshape(1,-1))
        action = np.argmax(Y[0])
    return action


In [10]:
1-0.001*500

0.5

In [13]:




gamma=0.99
epsilon_decay = 0.003
learning_rate =0.0001
max_e=1
min_e=0.0001
replay_batch_size=50
iterations=1000 

In [14]:
env = gym.make('LunarLander-v2')
state=env.reset()
online_model = initiate_NN(learning_rate)
memory=[]
episode_reward_rolling = []
episode_reward=[]

for episode in range(iterations):
    state=env.reset()
    current_episode_reward = 0
    done=False
    steps=0
    epsilon = min_e + (max_e - min_e)*np.exp(-epsilon_decay*episode)
    while not done:
        action = epsilon_action(epsilon)
        new_state, reward, done, info= env.step(action)
        steps+=1
        current_episode_reward +=reward
        memory.append([state,action,reward,new_state,done])
        state = new_state
        if len(memory)>=replay_batch_size:
            batch = random.sample(memory, replay_batch_size)
            batch = np.array(batch)
            non_terminal_samples = np.where(batch[:, 4] == False)
            y = np.copy(batch[:, 2])
            if len( non_terminal_samples[0]) > 0:
                #use online q values to select best action
                online_q = online_model.predict(np.vstack(batch[:, 3]))
                #use offline q values of best action as target 
                y[non_terminal_samples] += np.multiply(gamma, \
                            online_q[non_terminal_samples, \
                            np.argmax(online_q[non_terminal_samples, :][0], axis=1)][0])
            actions = np.array(batch[:, 1], dtype=int)
            y_target = online_model.predict(np.vstack(batch[:, 0]))
            y_target[range(replay_batch_size), actions] = y
            online_model.fit(np.vstack(batch[:, 0]), y_target, epochs=1, verbose=0)

    episode_reward.append(current_episode_reward) 
    if len(memory)>=10000:
        memory = memory[-10000:]
    if len(episode_reward)>=100:
        episode_reward_rolling.append(np.average(episode_reward[-100:]))
    else:
        episode_reward_rolling.append(np.average(episode_reward))
    if np.sum(np.array(episode_reward_rolling[-20:])>200)>19 and epsilon<0.1:
        break
    print('episode: ', episode, ' episode reward: ', '%.2f' % current_episode_reward, \
          ' rolling reward: ', '%.2f' % episode_reward_rolling[-1], ' Steps: ', steps, ' epsilon: ',\
          '%.2f' % epsilon)

episode:  0  episode reward:  -161.90  rolling reward:  -161.90  Steps:  113  epsilon:  1.00
episode:  1  episode reward:  -168.46  rolling reward:  -165.18  Steps:  66  epsilon:  1.00
episode:  2  episode reward:  -181.62  rolling reward:  -170.66  Steps:  113  epsilon:  0.99
episode:  3  episode reward:  -96.59  rolling reward:  -152.14  Steps:  87  epsilon:  0.99
episode:  4  episode reward:  -151.11  rolling reward:  -151.94  Steps:  73  epsilon:  0.99
episode:  5  episode reward:  -89.99  rolling reward:  -141.61  Steps:  73  epsilon:  0.99
episode:  6  episode reward:  -414.55  rolling reward:  -180.60  Steps:  115  epsilon:  0.98
episode:  7  episode reward:  -235.66  rolling reward:  -187.49  Steps:  92  epsilon:  0.98
episode:  8  episode reward:  -135.06  rolling reward:  -181.66  Steps:  103  epsilon:  0.98
episode:  9  episode reward:  -208.39  rolling reward:  -184.33  Steps:  104  epsilon:  0.97
episode:  10  episode reward:  -142.52  rolling reward:  -180.53  Steps:  72 

episode:  88  episode reward:  -142.79  rolling reward:  -158.99  Steps:  154  epsilon:  0.77
episode:  89  episode reward:  -131.43  rolling reward:  -158.69  Steps:  65  epsilon:  0.77
episode:  90  episode reward:  -93.50  rolling reward:  -157.97  Steps:  105  epsilon:  0.76
episode:  91  episode reward:  -172.99  rolling reward:  -158.13  Steps:  108  epsilon:  0.76
episode:  92  episode reward:  -231.27  rolling reward:  -158.92  Steps:  110  epsilon:  0.76
episode:  93  episode reward:  -146.60  rolling reward:  -158.79  Steps:  80  epsilon:  0.76
episode:  94  episode reward:  -188.18  rolling reward:  -159.10  Steps:  113  epsilon:  0.75
episode:  95  episode reward:  -139.04  rolling reward:  -158.89  Steps:  142  epsilon:  0.75
episode:  96  episode reward:  -173.27  rolling reward:  -159.04  Steps:  79  epsilon:  0.75
episode:  97  episode reward:  -68.61  rolling reward:  -158.12  Steps:  107  epsilon:  0.75
episode:  98  episode reward:  -192.39  rolling reward:  -158.46 

episode:  176  episode reward:  -117.06  rolling reward:  -109.10  Steps:  108  epsilon:  0.59
episode:  177  episode reward:  -13.91  rolling reward:  -108.36  Steps:  130  epsilon:  0.59
episode:  178  episode reward:  -62.00  rolling reward:  -107.97  Steps:  105  epsilon:  0.59
episode:  179  episode reward:  -73.40  rolling reward:  -107.53  Steps:  110  epsilon:  0.58
episode:  180  episode reward:  -222.48  rolling reward:  -107.69  Steps:  165  epsilon:  0.58
episode:  181  episode reward:  -113.35  rolling reward:  -106.88  Steps:  116  epsilon:  0.58
episode:  182  episode reward:  -55.64  rolling reward:  -106.80  Steps:  133  epsilon:  0.58
episode:  183  episode reward:  -78.05  rolling reward:  -105.87  Steps:  112  epsilon:  0.58
episode:  184  episode reward:  -61.70  rolling reward:  -105.19  Steps:  170  epsilon:  0.58
episode:  185  episode reward:  -68.18  rolling reward:  -104.68  Steps:  115  epsilon:  0.57
episode:  186  episode reward:  -58.08  rolling reward:  

episode:  265  episode reward:  -31.56  rolling reward:  -52.79  Steps:  180  epsilon:  0.45
episode:  266  episode reward:  -34.46  rolling reward:  -52.95  Steps:  251  epsilon:  0.45
episode:  267  episode reward:  46.00  rolling reward:  -52.41  Steps:  1000  epsilon:  0.45
episode:  268  episode reward:  28.40  rolling reward:  -51.00  Steps:  1000  epsilon:  0.45
episode:  269  episode reward:  -64.09  rolling reward:  -50.57  Steps:  145  epsilon:  0.45
episode:  270  episode reward:  16.97  rolling reward:  -49.69  Steps:  1000  epsilon:  0.44
episode:  271  episode reward:  -87.90  rolling reward:  -49.75  Steps:  113  epsilon:  0.44
episode:  272  episode reward:  74.69  rolling reward:  -47.68  Steps:  1000  epsilon:  0.44
episode:  273  episode reward:  68.58  rolling reward:  -46.62  Steps:  1000  epsilon:  0.44
episode:  274  episode reward:  -12.62  rolling reward:  -46.12  Steps:  137  epsilon:  0.44
episode:  275  episode reward:  -83.34  rolling reward:  -45.48  Steps

episode:  354  episode reward:  22.56  rolling reward:  1.60  Steps:  101  epsilon:  0.35
episode:  355  episode reward:  108.30  rolling reward:  3.07  Steps:  1000  epsilon:  0.34
episode:  356  episode reward:  45.46  rolling reward:  3.72  Steps:  85  epsilon:  0.34
episode:  357  episode reward:  90.26  rolling reward:  5.16  Steps:  1000  epsilon:  0.34
episode:  358  episode reward:  -33.48  rolling reward:  4.88  Steps:  169  epsilon:  0.34
episode:  359  episode reward:  2.29  rolling reward:  4.71  Steps:  120  epsilon:  0.34
episode:  360  episode reward:  -69.02  rolling reward:  4.49  Steps:  236  epsilon:  0.34
episode:  361  episode reward:  -215.38  rolling reward:  2.44  Steps:  385  epsilon:  0.34
episode:  362  episode reward:  34.00  rolling reward:  3.21  Steps:  1000  epsilon:  0.34
episode:  363  episode reward:  71.49  rolling reward:  5.65  Steps:  1000  epsilon:  0.34
episode:  364  episode reward:  -38.29  rolling reward:  4.78  Steps:  1000  epsilon:  0.34
e

episode:  444  episode reward:  107.69  rolling reward:  33.25  Steps:  1000  epsilon:  0.26
episode:  445  episode reward:  240.39  rolling reward:  35.12  Steps:  318  epsilon:  0.26
episode:  446  episode reward:  -5.38  rolling reward:  33.95  Steps:  122  epsilon:  0.26
episode:  447  episode reward:  109.85  rolling reward:  35.47  Steps:  1000  epsilon:  0.26
episode:  448  episode reward:  -156.96  rolling reward:  33.24  Steps:  244  epsilon:  0.26
episode:  449  episode reward:  192.44  rolling reward:  34.37  Steps:  411  epsilon:  0.26
episode:  450  episode reward:  -100.61  rolling reward:  32.37  Steps:  179  epsilon:  0.26
episode:  451  episode reward:  9.68  rolling reward:  34.34  Steps:  1000  epsilon:  0.26
episode:  452  episode reward:  176.62  rolling reward:  38.35  Steps:  596  epsilon:  0.26
episode:  453  episode reward:  121.84  rolling reward:  39.68  Steps:  1000  epsilon:  0.26
episode:  454  episode reward:  218.79  rolling reward:  41.65  Steps:  421  

episode:  533  episode reward:  236.78  rolling reward:  134.24  Steps:  302  epsilon:  0.20
episode:  534  episode reward:  -186.53  rolling reward:  132.90  Steps:  101  epsilon:  0.20
episode:  535  episode reward:  17.17  rolling reward:  131.57  Steps:  137  epsilon:  0.20
episode:  536  episode reward:  197.31  rolling reward:  133.30  Steps:  289  epsilon:  0.20
episode:  537  episode reward:  134.89  rolling reward:  134.42  Steps:  1000  epsilon:  0.20
episode:  538  episode reward:  260.58  rolling reward:  136.18  Steps:  510  epsilon:  0.20
episode:  539  episode reward:  248.12  rolling reward:  138.95  Steps:  764  epsilon:  0.20
episode:  540  episode reward:  -31.51  rolling reward:  136.89  Steps:  1000  epsilon:  0.20
episode:  541  episode reward:  234.51  rolling reward:  140.21  Steps:  313  epsilon:  0.20
episode:  542  episode reward:  56.40  rolling reward:  141.06  Steps:  141  epsilon:  0.20
episode:  543  episode reward:  187.28  rolling reward:  142.92  Step

episode:  622  episode reward:  202.01  rolling reward:  173.59  Steps:  508  epsilon:  0.15
episode:  623  episode reward:  -53.77  rolling reward:  170.56  Steps:  275  epsilon:  0.15
episode:  624  episode reward:  239.18  rolling reward:  170.17  Steps:  357  epsilon:  0.15
episode:  625  episode reward:  266.50  rolling reward:  170.77  Steps:  443  epsilon:  0.15
episode:  626  episode reward:  262.03  rolling reward:  170.83  Steps:  671  epsilon:  0.15
episode:  627  episode reward:  221.78  rolling reward:  171.68  Steps:  495  epsilon:  0.15
episode:  628  episode reward:  287.28  rolling reward:  173.62  Steps:  332  epsilon:  0.15
episode:  629  episode reward:  123.62  rolling reward:  173.92  Steps:  1000  epsilon:  0.15
episode:  630  episode reward:  238.00  rolling reward:  173.81  Steps:  379  epsilon:  0.15
episode:  631  episode reward:  221.46  rolling reward:  173.67  Steps:  462  epsilon:  0.15
episode:  632  episode reward:  254.55  rolling reward:  173.46  Step

episode:  711  episode reward:  70.77  rolling reward:  187.76  Steps:  1000  epsilon:  0.12
episode:  712  episode reward:  290.08  rolling reward:  188.06  Steps:  285  epsilon:  0.12
episode:  713  episode reward:  71.29  rolling reward:  186.35  Steps:  1000  epsilon:  0.12
episode:  714  episode reward:  206.51  rolling reward:  185.57  Steps:  659  epsilon:  0.12
episode:  715  episode reward:  242.39  rolling reward:  185.54  Steps:  392  epsilon:  0.12
episode:  716  episode reward:  273.43  rolling reward:  185.75  Steps:  524  epsilon:  0.12
episode:  717  episode reward:  269.68  rolling reward:  187.00  Steps:  248  epsilon:  0.12
episode:  718  episode reward:  214.75  rolling reward:  186.70  Steps:  867  epsilon:  0.12
episode:  719  episode reward:  280.46  rolling reward:  187.01  Steps:  273  epsilon:  0.12
episode:  720  episode reward:  195.85  rolling reward:  186.64  Steps:  474  epsilon:  0.12
episode:  721  episode reward:  268.51  rolling reward:  186.90  Steps

In [ ]:
online_model.save_weights('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.h5')

import pickle

with open('weights_singleDQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', 'wb') as fp:
    pickle.dump({'rewards':episode_reward,'rolling_reward':episode_reward_rolling}, fp)


In [167]:
rward = pickle.load( open( 'weights_DQN_'+str(max_e)+'-'+str(min_e)+'-'+str(epsilon_decay)\
                         +'-'+str(gamma)+'-'+str(replay_batch_size)+'-'+str(learning_rate)+'.json', "rb" ) )

In [168]:
rward

{'rewards': [-131.3223570990876, -248.23258991485326, -133.48636655895385],
 'rolling_reward': [-131.3223570990876,
  -189.77747350697044,
  -171.01377119096492]}

In [163]:
episode_reward

[]

In [152]:
Y_target.shape

(5, 4)

In [142]:
batch[terminal_samples].shape

(0, 5)

In [125]:



# i:index for sample [state,action,reward,new_state,done]
for i,sample in enumerate(batch):
    terminal_sample = sample[4]
    new_state_sample = sample[3]
    state_sample = sample[0]
    X[i]=state_sample
    action_sample = sample[1]
    Y_target[i] = online_model.predict(state_sample.reshape(1,-1))[0]
    if terminal:
        #terminal then targetY is reward
        Y_target[i][action_sample] = sample[2]
    else:
        #use online weights to select argmax action under new_state

        online_qvalues_batch =online_model.predict(new_state_sample.reshape(1,-1)) 
        max_action = np.argmax(online_qvalues_batch)
        #use target weights to calculate Qvalue
        #non-terminal then targetY is reward +gamma*new Qvalue
        Y_target[i][action_sample] = sample[2] + gamma * target_model.predict(new_state_sample.reshape(1,-1)).reshape(4,)[max_action]

online_model.fit(np.array(X), np.array(Y_target), verbose=0)


100

In [ ]:
Notice that the selection of the action, in the argmax, is


In [91]:
online_qvalues_batch

[array([[ 0.18740644, -0.2721934 ,  0.2233217 , -0.45678362]],
       dtype=float32),
 array([[ 0.19613695, -0.27545747,  0.20470577, -0.46681872]],
       dtype=float32),
 array([[ 0.18746991, -0.25133634,  0.19952038, -0.4359238 ]],
       dtype=float32),
 array([[ 0.19334945, -0.2700162 ,  0.19816679, -0.45707738]],
       dtype=float32),
 array([[ 0.19308917, -0.27386588,  0.21208042, -0.46272066]],
       dtype=float32)]

In [113]:
np.array(X)

array([[-0.00213823,  1.4003743 , -0.21659775, -0.4687014 ,  0.0024845 ,
         0.04906261,  0.        ,  0.        ],
       [-0.00846777,  1.3651963 , -0.20548455, -0.5481141 ,  0.00758275,
         0.00502167,  0.        ,  0.        ],
       [-0.01887293,  1.3004851 , -0.22156799, -0.61512053,  0.00625335,
         0.04479944,  0.        ,  0.        ],
       [-0.01250668,  1.3402183 , -0.20310256, -0.5355056 ,  0.00488083,
        -0.02924636,  0.        ,  0.        ],
       [-0.00641537,  1.3775289 , -0.21631236, -0.5210343 ,  0.00733169,
         0.04846335,  0.        ,  0.        ]], dtype=float32)

In [112]:
Y_target

[array([-1.2552428 , -0.2741652 ,  0.2263453 , -0.45686692], dtype=float32),
 array([ 0.19308917, -0.27386588,  0.21208042, -0.6722371 ], dtype=float32),
 array([ 0.1933487, -0.2546284,  5.1014786, -0.445849 ], dtype=float32),
 array([-0.91460276, -0.27277392,  0.20281965, -0.4571485 ], dtype=float32),
 array([ 0.18904   , -0.27001685,  0.21992686, -0.7534547 ], dtype=float32)]

In [ ]:

class DDQL:
    def __init__(self, nS, nA):
        self.nS = nS
        self.nA = nA
        self.epsilon = 1
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.9993
        self.gamma = 0.99
        self.learning_rate = 0.0001
        self.epochs = 1
        self.verbose = 0
        self.minibatch_size = 30
        self.memory = deque(maxlen=5000)
        self.model = self.create_model()
        self.target_model = self.create_model()

    def create_model(self):
        model = Sequential()

        # Add 2 hidden layers with 64 nodes each
        model.add(Dense(64, input_dim=self.nS, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(self.nA, activation='linear'))
        model.compile(loss='mse', optimizer=Adam(lr=self.learning_rate))
        return model

    def add_memory(self, s, a, r, s_prime, done):
        self.memory.append((s, a, r, s_prime, done))

    def target_model_update(self):
        self.target_model.set_weights(self.model.get_weights())

    def selectAction(self, s):
        if np.random.rand() <= self.epsilon:
            return np.random.choice(self.nA)
        q = self.model.predict(s)
        return np.argmax(q[0])

    def replay(self):
        # Vectorized method for experience replay
        minibatch = random.sample(self.memory, self.minibatch_size)
        minibatch = np.array(minibatch)
        not_done_indices = np.where(minibatch[:, 4] == False)
        y = np.copy(minibatch[:, 2])

        # If minibatch contains any non-terminal states, use separate update rule for those states
        if len(not_done_indices[0]) > 0:
            predict_sprime = self.model.predict(np.vstack(minibatch[:, 3]))
            predict_sprime_target = self.target_model.predict(np.vstack(minibatch[:, 3]))
            
            # Non-terminal update rule
            y[not_done_indices] += np.multiply(self.gamma, \
                        predict_sprime_target[not_done_indices, \
                        np.argmax(predict_sprime[not_done_indices, :][0], axis=1)][0])

        actions = np.array(minibatch[:, 1], dtype=int)
        y_target = self.model.predict(np.vstack(minibatch[:, 0]))
        y_target[range(self.minibatch_size), actions] = y
        self.model.fit(np.vstack(minibatch[:, 0]), y_target, epochs=self.epochs, verbose=self.verbose)

    def replayIterative(self):
        # Iterative method - this performs the same function as replay() but is not vectorized 
        s_list = []
        y_state_list = []
        minibatch = random.sample(self.memory, self.minibatch_size)
        for s, a, r, s_prime, done in minibatch:
            s_list.append(s)
            y_action = r
            if not done:
                y_action = r + self.gamma * np.amax(self.model.predict(s_prime)[0])

            print(y_action)
            
            y_state = self.model.predict(s)
            y_state[0][a] = y_action
            y_state_list.append(y_state)
        self.model.fit(np.squeeze(s_list), np.squeeze(y_state_list), batch_size=self.minibatch_size, epochs=1, verbose=0)

if __name__ == "__main__":
    np.set_printoptions(precision=2)
    env = gym.make('LunarLander-v2')
    env = wrappers.Monitor(env, tdir, force=True, video_callable=False)

    nS = env.observation_space.shape[0]
    nA = env.action_space.n

    agent = DDQL(nS, nA)

    # Set to true to use saved model
    episodes = 10000

    # Cumulative reward
    reward_avg = deque(maxlen=100)

    for e in range(episodes):
        episode_reward = 0
        s = env.reset()
        s = np.reshape(s, [1, nS])

        for time in range(1000):

            # Query next action from learner and perform action
            a = agent.selectAction(s)
            s_prime, r, done, info = env.step(a)

            # Add cumulative reward
            episode_reward += r

            # Reshape new state
            s_prime = np.reshape(s_prime, [1, nS])

            # Add experience to memory
            agent.add_memory(s, a, r, s_prime, done)

            # Set current state to new state
            s = s_prime

            #Perform experience replay if memory length is greater than minibatch length
   
            if len(agent.memory) > agent.minibatch_size:
                agent.replay()

            # If episode is done, exit loop
            if done:
                agent.target_model_update()
                break


        # epsilon decay
        if agent.epsilon > agent.epsilon_min:
            agent.epsilon *= agent.epsilon_decay

        # Running average of past 100 episodes
        reward_avg.append(episode_reward)
        print('episode: ', e, ' score: ', '%.2f' % episode_reward, ' avg_score: ', '%.2f' % np.average(reward_avg), ' frames: ', time, ' epsilon: ', '%.2f' % agent.epsilon)
        
        # with open('trained_agent.txt', 'a') as f:
        #     f.write(str(np.average(reward_avg)) + '\n')

    env.close()
#     gym.upload(tdir, api_key='sk_EJo79Jo0RsSEA4EF6Cp5mg')


#agent.model.save_weights('weights.h5')
#agent.model.load_weights('weights.h5')

episode:  0  score:  -101.20  avg_score:  -101.20  frames:  104  epsilon:  1.00
episode:  1  score:  -147.99  avg_score:  -124.59  frames:  74  epsilon:  1.00
episode:  2  score:  -95.67  avg_score:  -114.95  frames:  74  epsilon:  1.00
episode:  3  score:  -218.30  avg_score:  -140.79  frames:  94  epsilon:  1.00
episode:  4  score:  -231.98  avg_score:  -159.03  frames:  86  epsilon:  1.00
episode:  5  score:  -161.32  avg_score:  -159.41  frames:  68  epsilon:  1.00
episode:  6  score:  -96.64  avg_score:  -150.44  frames:  66  epsilon:  1.00
episode:  7  score:  -141.43  avg_score:  -149.32  frames:  97  epsilon:  0.99
episode:  8  score:  -480.81  avg_score:  -186.15  frames:  102  epsilon:  0.99
episode:  9  score:  -371.18  avg_score:  -204.65  frames:  92  epsilon:  0.99
episode:  10  score:  -161.56  avg_score:  -200.73  frames:  95  epsilon:  0.99
episode:  11  score:  -80.20  avg_score:  -190.69  frames:  61  epsilon:  0.99
episode:  12  score:  -332.47  avg_score:  -201.60 

episode:  103  score:  -168.96  avg_score:  -179.65  frames:  118  epsilon:  0.93
episode:  104  score:  -64.43  avg_score:  -177.97  frames:  91  epsilon:  0.93
episode:  105  score:  -200.70  avg_score:  -178.36  frames:  72  epsilon:  0.93
episode:  106  score:  32.45  avg_score:  -177.07  frames:  121  epsilon:  0.93
episode:  107  score:  -260.25  avg_score:  -178.26  frames:  117  epsilon:  0.93
episode:  108  score:  -110.97  avg_score:  -174.56  frames:  80  epsilon:  0.93
episode:  109  score:  -253.15  avg_score:  -173.38  frames:  94  epsilon:  0.93
episode:  110  score:  -125.12  avg_score:  -173.02  frames:  88  epsilon:  0.93
episode:  111  score:  -189.74  avg_score:  -174.11  frames:  72  epsilon:  0.92
episode:  112  score:  -140.07  avg_score:  -172.19  frames:  94  epsilon:  0.92
episode:  113  score:  -304.20  avg_score:  -173.17  frames:  71  epsilon:  0.92
episode:  114  score:  -128.94  avg_score:  -173.02  frames:  72  epsilon:  0.92
episode:  115  score:  -294.

episode:  204  score:  -298.51  avg_score:  -150.87  frames:  97  epsilon:  0.87
episode:  205  score:  -77.35  avg_score:  -149.64  frames:  146  epsilon:  0.87
episode:  206  score:  -124.22  avg_score:  -151.21  frames:  68  epsilon:  0.87
episode:  207  score:  -40.81  avg_score:  -149.01  frames:  144  epsilon:  0.86
episode:  208  score:  -83.08  avg_score:  -148.73  frames:  67  epsilon:  0.86
episode:  209  score:  -268.33  avg_score:  -148.89  frames:  92  epsilon:  0.86
episode:  210  score:  -124.07  avg_score:  -148.88  frames:  105  epsilon:  0.86
episode:  211  score:  -123.48  avg_score:  -148.21  frames:  122  epsilon:  0.86
episode:  212  score:  -143.46  avg_score:  -148.25  frames:  80  epsilon:  0.86
episode:  213  score:  -86.07  avg_score:  -146.07  frames:  96  epsilon:  0.86
episode:  214  score:  -86.35  avg_score:  -145.64  frames:  94  epsilon:  0.86
episode:  215  score:  -55.86  avg_score:  -143.25  frames:  93  epsilon:  0.86
episode:  216  score:  -58.61 